# Running Jobs on Hadoop

The aim of this exercise is to get familiar with Hadoop. We will show how to run Hadoop application and create our own Python-based streaming application for parsing log file data.

## Configure Hadoop Environment and Test Applications

For the following exercise we will use two examples provided as part of the standard Hadoop distribution. We use the Hortonworks HDP 2.3.2 deployed on Amazon Web Services (EC2). First we need to set these two variables to the `jar` files containing the application.

In [4]:
HADOOP_EXAMPLES="/usr/hdp/2.3.2.0-2950/hadoop-mapreduce/hadoop-mapreduce-examples.jar"
HADOOP_STREAMING="/usr/hdp/2.3.2.0-2950/hadoop-mapreduce/hadoop-streaming.jar"

## 1. Hadoop Services (HDFS and YARN)

In [5]:
!hadoop dfsadmin -report

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Configured Capacity: 425150103552 (395.95 GB)
Present Capacity: 392212938752 (365.28 GB)
DFS Remaining: 389594132480 (362.84 GB)
DFS Used: 2618806272 (2.44 GB)
DFS Used%: 0.67%
Under replicated blocks: 4
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 0

-------------------------------------------------
report: Access denied for user radical. Superuser privilege is required


In [6]:
!yarn node -list all

15/11/08 16:43:39 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/08 16:43:39 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
Total Nodes:4
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
ip-10-63-179-69.ec2.internal:45454	        RUNNING	ip-10-63-179-69.ec2.internal:8042	                           0
ip-10-145-0-4.ec2.internal:45454	        RUNNING	ip-10-145-0-4.ec2.internal:8042	                           1
ip-10-218-164-206.ec2.internal:45454	        RUNNING	ip-10-218-164-206.ec2.internal:8042	                           1
ip-10-179-174-236.ec2.internal:45454	        RUNNING	ip-10-179-174-236.ec2.internal:8042	                           1


## 2. Terasort

In [15]:
!hdfs dfs -rm -r teragen

15/11/08 16:44:57 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 360 minutes, Emptier interval = 0 minutes.
Moved: 'hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/teragen' to trash at: hdfs://ip-10-63-179-69.ec2.internal:8020/user/radical/.Trash/Current


In [16]:
!yarn jar $HADOOP_EXAMPLES teragen 100000 teragen

15/11/08 16:45:00 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/08 16:45:00 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/08 16:45:01 INFO terasort.TeraSort: Generating 100000 using 2
15/11/08 16:45:01 INFO mapreduce.JobSubmitter: number of splits:2
15/11/08 16:45:01 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1447000128355_0005
15/11/08 16:45:01 INFO impl.YarnClientImpl: Submitted application application_1447000128355_0005
15/11/08 16:45:01 INFO mapreduce.Job: The url to track the job: http://ip-10-63-179-69.ec2.internal:8088/proxy/application_1447000128355_0005/
15/11/08 16:45:01 INFO mapreduce.Job: Running job: job_1447000128355_0005
15/11/08 16:45:07 INFO mapreduce.Job: Job job_1447000128355_0005 running in uber mode : false
15/11/08 16:45:07 INFO mapreduce.Job:  map 0% reduce 0%
15/11/08 16:45:14 INFO mapreduce.Job:  map 50% reduce 0

In [8]:
!yarn jar $HADOOP_EXAMPLES terasort teragen teraout

15/11/08 16:43:43 INFO terasort.TeraSort: starting
15/11/08 16:43:45 INFO input.FileInputFormat: Total input paths to process : 2
Spent 137ms computing base-splits.
Spent 2ms computing TeraScheduler splits.
Computing input splits took 140ms
Sampling 2 splits of 2
Making 1 from 100000 sampled records
Computing parititions took 394ms
Spent 537ms computing partitions.
15/11/08 16:43:45 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/08 16:43:46 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/08 16:43:46 INFO mapreduce.JobSubmitter: number of splits:2
15/11/08 16:43:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1447000128355_0003
15/11/08 16:43:47 INFO impl.YarnClientImpl: Submitted application application_1447000128355_0003
15/11/08 16:43:47 INFO mapreduce.Job: The url to track the job: http://ip-10-63-179-69.ec2.internal:8088/proxy/application

## 3. Word Count

Count the words contained in the log file located at `/data/nasa/NASA_access_log_Jul95`

In [9]:
!hdfs dfs -rm -r wordcount-out

rm: `wordcount-out': No such file or directory


In [10]:
!hdfs dfs -text /data/nasa/NASA_access_log_Jul95 | head

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 

In [11]:
!yarn jar $HADOOP_EXAMPLES wordcount /data/nasa/ wordcount-out/

15/11/08 16:44:14 INFO impl.TimelineClientImpl: Timeline service address: http://ip-10-63-179-69.ec2.internal:8188/ws/v1/timeline/
15/11/08 16:44:15 INFO client.RMProxy: Connecting to ResourceManager at ip-10-63-179-69.ec2.internal/10.63.179.69:8050
15/11/08 16:44:15 INFO input.FileInputFormat: Total input paths to process : 1
15/11/08 16:44:15 INFO mapreduce.JobSubmitter: number of splits:2
15/11/08 16:44:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1447000128355_0004
15/11/08 16:44:16 INFO impl.YarnClientImpl: Submitted application application_1447000128355_0004
15/11/08 16:44:16 INFO mapreduce.Job: The url to track the job: http://ip-10-63-179-69.ec2.internal:8088/proxy/application_1447000128355_0004/
15/11/08 16:44:16 INFO mapreduce.Job: Running job: job_1447000128355_0004
15/11/08 16:44:22 INFO mapreduce.Job: Job job_1447000128355_0004 running in uber mode : false
15/11/08 16:44:22 INFO mapreduce.Job:  map 0% reduce 0%
15/11/08 16:44:34 INFO mapreduce.Job:  map 6

## 4. Log Parsing

Use the commands `head`, `cat`, `uniq`, `wc`, `sort`, `find`, `xargs`, `awk` to evaluate the NASA log file:

Which page was called the most?
What was the most frequent return code?
How many errors occurred? What is the percentage of errors?
Implement a Python version of this Unix Shell script using this script as template!

Run the Python script inside an Hadoop Streaming job.

In [12]:
!cat mapreduce_streaming.py 

cat: mapreduce_streaming.py: No such file or directory


In [18]:
!yarn jar $HADOOP_STREAMING -input /data/nasa -output logs-parsed \
                            -files mapreduce_streaming.py \
                            -mapper "python mapreduce_streaming.py map" \
                            -reducer "python mapreduce_streaming.py reduce" 

15/11/08 16:47:11 ERROR streaming.StreamJob: Unrecognized option: -files
Usage: $HADOOP_PREFIX/bin/hadoop jar hadoop-streaming.jar [options]
Options:
  -input          <path> DFS input file(s) for the Map step.
  -output         <path> DFS output directory for the Reduce step.
  -mapper         <cmd|JavaClassName> Optional. Command to be run as mapper.
  -combiner       <cmd|JavaClassName> Optional. Command to be run as combiner.
  -reducer        <cmd|JavaClassName> Optional. Command to be run as reducer.
  -file           <file> Optional. File/dir to be shipped in the Job jar file.
                  Deprecated. Use generic option "-files" instead.
  -inputformat    <TextInputFormat(default)|SequenceFileAsTextInputFormat|JavaClassName>
                  Optional. The input format class.
  -outputformat   <TextOutputFormat(default)|JavaClassName>
                  Optional. The output format class.
  -partitioner    <JavaClassName>  Optional. The partitioner class.
  -numReduceTasks <n

In [17]:
!hdfs dfs -ls logs-parsed

ls: `logs-parsed': No such file or directory
